## Class Data Construction


#### Table of Contents

* [Overview](#Overview)
* [Set Up](#Set-Up)
* [GDP](#GDP)
* [Unemployment](#Unemployment)
* [BDS](#BDS)
* [CDC](#CDC)
* [Social Explorer](#Social-Explorer)
* [Saving](#Saving)

# Overview

This notebook constructs the data set used for in-class examples and for the workshops.
The data is at the county-year level from 2002-2018.
We will be constructing three different labels that will demonstrate different techniques.

* Labels
    1. Continuous - %$\Delta$rGDP
    2. Binary - $I\{net job creation \geq 0 \}$
    3. Multiclass - County unemployment rate $\in \{\ll U\_rate_{national}, \sim U\_rate_{national}, \gg  U\_rate_{national}\}$
* Feautres
    1. Population
    2. Population density
    3. Percent black
    4. Percent hispanic
    5. Establishment entry rate
    6. Establishment exit rate
    7. Employment per establishments
    8. Quasi LF participation rate
    9. Year Fixed Effects


The data are sourced from five government databases:

1. [BEA](https://apps.bea.gov/regional/downloadzip.cfm) - CAGDP9
2. [BLS Local Area Unemployment Statistics (LAUS)](https://www.bls.gov/lau/tables.htm)  - County Data
3. [Census Business Dynamic Statistics (BDS)](https://www.census.gov/data/tables/time-series/econ/bds/bds-tables.html) - One-Way Tables: County
4. [CDC WONDER](https://wonder.cdc.gov/Bridged-Race-v2019.HTML) - Demographics
5. [Social Explorer](http://www.library.illinois.edu/proxy/go.php?url=http://www.socialexplorer.com/ezproxy) - County land area


***

# Set Up

In [77]:
import pandas as pd
import numpy as np
import os

In [78]:
os.chdir('C:/Users/hubst/ECON_490/class data')
print(os.listdir())

['.ipynb_checkpoints', 'area codebook.txt', 'area.csv', 'bds2018_cty.csv', 'CAGDP9__ALL_AREAS_2001_2019.csv', 'Class Data Preamble.ipynb', 'class_data.pkl', 'demographics', 'unemployment']


***
# GDP

In [79]:
gdp = pd.read_csv('CAGDP9__ALL_AREAS_2001_2019.csv', encoding = 'ANSI')

gdp.head(3)

E:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,2001,2002,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,"""00000""",United States,,CAGDP9,1.0,...,All industry total,Thousands of chained 2012 dollars,13262079000,13493064000,...,15598753000,15840664000,16197007000,16495369000,16912038000,17432170000,17730509000,18144105000,18687786000,19091662000
1,"""00000""",United States,,CAGDP9,2.0,...,Private industries,Thousands of chained 2012 dollars,11465144000,11669109000,...,13467315000,13697390000,14037519000,14302267000,14715952000,15232954000,15511037000,15902123000,16415475000,16804174000
2,"""00000""",United States,,CAGDP9,3.0,11,"Agriculture, forestry, fishing and hunting",Thousands of chained 2012 dollars,156155000,161617000,...,193856000,186395000,179573000,209388000,211758000,227217000,238834000,233675000,243393000,243607000


In [80]:
gdp.dtypes

GeoFIPS                    object
GeoName                    object
Region                     object
TableName                  object
LineCode                  float64
IndustryClassification     object
Description                object
Unit                       object
2001                       object
2002                       object
2003                       object
2004                       object
2005                       object
2006                       object
2007                       object
2008                       object
2009                       object
2010                       object
2011                       object
2012                       object
2013                       object
2014                       object
2015                       object
2016                       object
2017                       object
2018                       object
2019                       object
dtype: object

In [81]:
print(gdp['2001'].drop_duplicates().sort_values().head(10), '\n')
print(gdp['2001'].drop_duplicates().sort_values(ascending = False).head(10), '\n')

75          (D)
2686       (NA)
241           0
9565          1
31284        10
15357       100
9714       1000
58791     10000
61409    100003
69544    100005
Name: 2001, dtype: object 

48312       99996
66557        9999
100165    9997966
20448       99975
981         99974
40743        9997
22178       99969
49787        9996
95466      999581
49583       99956
Name: 2001, dtype: object 



In [82]:
gdp.replace('(NA)', np.nan, inplace = True)
gdp.replace('(D)', np.nan, inplace = True)
gdp.dropna(inplace = True)

In [83]:
gdp = gdp[gdp['LineCode'] == 1].reset_index(drop = True)

In [84]:
gdp.head(1)

,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,2001,2002,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,"""00000""",United States,,CAGDP9,1.0,...,All industry total,Thousands of chained 2012 dollars,13262079000,13493064000,...,15598753000,15840664000,16197007000,16495369000,16912038000,17432170000,17730509000,18144105000,18687786000,19091662000


In [85]:
gdp['fips'] = gdp['GeoFIPS'].str[-6:-1]

In [86]:
gdp = gdp[gdp['fips'].str[2:5] != '000']

In [87]:
gdp['fips'] = pd.to_numeric(gdp.fips)

In [88]:
gdp.shape

(3107, 28)

In [89]:
old = [str(x) for x in range(2001, 2020)]
new = ['gdp' + str(x) for x in range(2001, 2020)]
gdp.rename(columns = dict(zip(old, new)), inplace = True)
gdp

,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,gdp2001,gdp2002,...,gdp2011,gdp2012,gdp2013,gdp2014,gdp2015,gdp2016,gdp2017,gdp2018,gdp2019,fips
2,"""01001""","Autauga, AL",5,CAGDP9,1.0,...,All industry total,Thousands of chained 2012 dollars,943367,973575,...,1323275,1487130,1416052,1412808,1529517,1581893,1494278,1537059,1501769,1001
3,"""01003""","Baldwin, AL",5,CAGDP9,1.0,...,All industry total,Thousands of chained 2012 dollars,3950240,4148592,...,4934067,5051126,5180403,5258587,5437402,5625966,5754579,6006401,6140514,1003
4,"""01005""","Barbour, AL",5,CAGDP9,1.0,...,All industry total,Thousands of chained 2012 dollars,812861,810625,...,745563,723755,818289,769252,768307,739860,743441,771536,762856,1005
5,"""01007""","Bibb, AL",5,CAGDP9,1.0,...,All industry total,Thousands of chained 2012 dollars,296728,304939,...,381749,371222,374588,367880,353397,355874,355890,363268,389547,1007
6,"""01009""","Blount, AL",5,CAGDP9,1.0,...,All industry total,Thousands of chained 2012 dollars,810664,816704,...,792710,809863,859882,854532,874969,829566,876376,896074,869049,1009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3154,"""56037""","Sweetwater, WY",7,CAGDP9,1.0,...,All industry total,Thousands of chained 2012 dollars,3859184,3377438,...,4217027,4135106,4130223,3910288,3840139,3907803,3835234,3769844,3861073,56037
3155,"""56039""","Teton, WY",7,CAGDP9,1.0,...,All industry total,Thousands of chained 2012 dollars,1565419,1624363,...,1734117,1800844,1845841,1982448,2027554,2053838,2125624,2193369,2250191,56039
3156,"""56041""","Uinta, WY",7,CAGDP9,1.0,...,All industry total,Thousands of chained 2012 dollars,1003675,1102392,...,1027539,1000607,969598,948666,957914,951046,923383,927914,926327,56041
3157,"""56043""","Washakie, WY",7,CAGDP9,1.0,...,All industry total,Thousands of chained 2012 dollars,348016,340844,...,399244,370923,376328,385128,396009,358659,341419,352737,369344,56043


In [90]:
df = pd.wide_to_long(gdp, stubnames = ['gdp'], i = 'fips', j = 'year')
df

,,IndustryClassification,GeoName,Unit,Description,LineCode,TableName,Region,GeoFIPS,gdp
fips,year,,,,,,,,,
1001,2001,...,"Autauga, AL",Thousands of chained 2012 dollars,All industry total,1.0,CAGDP9,5,"""01001""",943367
1003,2001,...,"Baldwin, AL",Thousands of chained 2012 dollars,All industry total,1.0,CAGDP9,5,"""01003""",3950240
1005,2001,...,"Barbour, AL",Thousands of chained 2012 dollars,All industry total,1.0,CAGDP9,5,"""01005""",812861
1007,2001,...,"Bibb, AL",Thousands of chained 2012 dollars,All industry total,1.0,CAGDP9,5,"""01007""",296728
1009,2001,...,"Blount, AL",Thousands of chained 2012 dollars,All industry total,1.0,CAGDP9,5,"""01009""",810664
...,...,...,...,...,...,...,...,...,...,...
56037,2019,...,"Sweetwater, WY",Thousands of chained 2012 dollars,All industry total,1.0,CAGDP9,7,"""56037""",3861073
56039,2019,...,"Teton, WY",Thousands of chained 2012 dollars,All industry total,1.0,CAGDP9,7,"""56039""",2250191
56041,2019,...,"Uinta, WY",Thousands of chained 2012 dollars,All industry total,1.0,CAGDP9,7,"""56041""",926327


In [74]:
df = df[['GeoName', 'gdp']]

In [75]:
df['gdp'] = df.gdp.astype(int)

$$
\% \Delta = \frac{new - old}{old}100\%
$$

In [76]:
df['gdp_old'] = df['gdp'].groupby('fips').shift(1)
df['pct_d_rgdp'] = (  df['gdp'] - df['gdp_old']  )/df['gdp_old']*100
df
#df.head()

,,GeoName,gdp,gdp_old,pct_d_rgdp
fips,year,,,,
1001,2001,"Autauga, AL",943367,NaN,NaN
1003,2001,"Baldwin, AL",3950240,NaN,NaN
1005,2001,"Barbour, AL",812861,NaN,NaN
1007,2001,"Bibb, AL",296728,NaN,NaN
1009,2001,"Blount, AL",810664,NaN,NaN
...,...,...,...,...,...
56037,2019,"Sweetwater, WY",3861073,3769844.0,2.419968
56039,2019,"Teton, WY",2250191,2193369.0,2.590627
56041,2019,"Uinta, WY",926327,927914.0,-0.171029


In [56]:
df = df[['GeoName', 'pct_d_rgdp', 'gdp', 'gdp_old']].dropna().sort_index()
df.head()

GeoName  pct_d_rgdp      gdp    gdp_old
fips year                                             
1001 2002  Autauga, AL    3.202147   973575   943367.0
     2003  Autauga, AL    1.434404   987540   973575.0
     2004  Autauga, AL   15.061365  1136277   987540.0
     2005  Autauga, AL    0.333105  1140062  1136277.0
     2006  Autauga, AL    7.440034  1224883  1140062.0

In [57]:
df

GeoName  pct_d_rgdp      gdp    gdp_old
fips  year                                             
1001  2002  Autauga, AL    3.202147   973575   943367.0
      2003  Autauga, AL    1.434404   987540   973575.0
      2004  Autauga, AL   15.061365  1136277   987540.0
      2005  Autauga, AL    0.333105  1140062  1136277.0
      2006  Autauga, AL    7.440034  1224883  1140062.0
...                 ...         ...      ...        ...
56045 2015   Weston, WY   12.558802   322302   286341.0
      2016   Weston, WY  -10.381257   288843   322302.0
      2017   Weston, WY   -0.153371   288400   288843.0
      2018   Weston, WY    7.624480   310389   288400.0
      2019   Weston, WY    5.001466   325913   310389.0

[55926 rows x 4 columns]

***
# Unemployment

In [19]:
files = os.listdir('unemployment')

In [20]:
urate = pd.read_excel('unemployment/' + files[0], engine = 'openpyxl',
              skiprows = 5, skipfooter = 3,
              names = ['lau_code', 'fips_state', 'fips_county', 'name', 'year', 'na',
                       'lf', 'emp', 'unemp', 'urate'])

urate = urate[urate['fips_state'] <= 56]

# create FIPS variable

urate['fips'] = urate.fips_state*1000 + urate.fips_county

urate['urate_nat'] = sum(urate['unemp'])/sum(urate['lf'])*100
urate['urate_diff'] = urate.urate - urate.urate_nat
urate.urate_diff.describe()

urate['urate_bin'] = np.where(urate['urate_diff'] < -0.5, 'lower',
                 np.where(urate['urate_diff'] <= 0.5, 'similar', 'higher'))

# What percent in which bin
print(urate['urate_bin'].value_counts()/urate.shape[0]*100)

urate = urate[['fips', 'year', 'urate_bin', 'lf']]

lower      43.835616
higher     33.290857
similar    22.873527
Name: urate_bin, dtype: float64


In [21]:
temp = pd.DataFrame(columns = ['fips', 'year', 'urate_bin', 'lf'])

for file in files:
    print(file)
    urate = pd.read_excel('unemployment/' + file, engine = 'openpyxl',
              skiprows = 5, skipfooter = 3,
              names = ['lau_code', 'fips_state', 'fips_county', 'name', 'year', 'na',
                       'lf', 'emp', 'unemp', 'urate'])

    urate = urate[urate['fips_state'] <= 56]

    # create FIPS variable

    urate['fips'] = urate.fips_state*1000 + urate.fips_county
    
    urate['urate_nat'] = sum(urate['unemp'])/sum(urate['lf'])*100
    urate['urate_diff'] = urate.urate - urate.urate_nat

    urate['urate_bin'] = np.where(urate['urate_diff'] < -0.5, 'lower',
                 np.where(urate['urate_diff'] <= 0.5, 'similar', 'higher'))

    urate = urate[['fips', 'year', 'urate_bin', 'lf']]
    temp = temp.append(urate)

laucnty02.xlsx
laucnty03.xlsx
laucnty04.xlsx
laucnty05.xlsx
laucnty06.xlsx
laucnty07.xlsx
laucnty08.xlsx
laucnty09.xlsx
laucnty10.xlsx
laucnty11.xlsx
laucnty12.xlsx
laucnty13.xlsx
laucnty14.xlsx
laucnty15.xlsx
laucnty16.xlsx
laucnty17.xlsx
laucnty18.xlsx


In [22]:
temp.isnull().any()

fips         False
year         False
urate_bin    False
lf           False
dtype: bool

In [23]:
# temp = temp.set_index(['fips', 'year'])
temp.sort_index(inplace = True)
temp.head(2)

,fips,year,urate_bin,lf
0,1001,2002,lower,22094
0,1001,2002,lower,22094


In [24]:
df = df.join(temp, how = 'inner')

ValueError: cannot join with no overlapping index names

***
# BDS

In [ ]:
bds = pd.read_csv('bds2018_cty.csv', na_values = '(D)').dropna()

In [ ]:
bds = bds[(bds['year'] >= 2002) & (bds['year'] <= 2018) & (bds.cty < 998)].copy()

In [ ]:
bds['fips'] = bds['st']*1000 + bds['cty']
bds['pos_net_jobs'] = (bds['net_job_creation'] >= 0)*1

In [ ]:
bds['emp_estabs'] = bds['emp']/bds['estabs']

In [ ]:
bds = bds[['fips', 'year', 'pos_net_jobs', 'emp_estabs', 'estabs_entry_rate',
    'estabs_exit_rate']].set_index(['fips', 'year']).sort_index()

In [ ]:
df = df.join(bds, how = 'inner')

***
# CDC

In [ ]:
os.listdir('demographics')

In [ ]:
pop = pd.read_table('demographics/population.txt')
pop.head(1)

In [ ]:
pop = pop.drop(columns = ['Notes', 'County', 'Yearly July 1st Estimates Code']
        ).dropna().rename(columns = {'County Code': 'fips',
                                    'Yearly July 1st Estimates': 'year', 
                                    'Population': 'pop'}
                         ).set_index(['fips', 'year'])

In [ ]:
df = df.join(pop)

In [ ]:
black = pd.read_table('demographics/black.txt')
black = black.drop(columns = ['Notes', 'County', 'Yearly July 1st Estimates Code']
        ).dropna().rename(columns = {'County Code': 'fips',
                                    'Yearly July 1st Estimates': 'year', 
                                    'Population': 'pop'}
                         ).set_index(['fips', 'year'])
df = df.join(black)

In [ ]:
hisp = pd.read_table('demographics/hispanic.txt')
hisp = hisp.drop(columns = ['Notes', 'County', 'Yearly July 1st Estimates Code']
        ).dropna().rename(columns = {'County Code': 'fips',
                                    'Yearly July 1st Estimates': 'year', 
                                    'Population': 'pop'}
                         ).set_index(['fips', 'year'])
df = df.join(hisp)

In [ ]:
wrk_age = pd.read_table('demographics/working age pop.txt')
wrk_age = wrk_age.drop(columns = ['Notes', 'County', 'Yearly July 1st Estimates Code']
        ).dropna().rename(columns = {'County Code': 'fips',
                                    'Yearly July 1st Estimates': 'year', 
                                    'Population': 'pop'}
                         ).set_index(['fips', 'year'])
df = df.join(wrk_age)

In [ ]:
df['pop_pct_black']  = df['black']/df['pop']*100
df['pop_pct_hisp']  = df['hisp']/df['pop']*100
df['lfpr']  = df['lf']/df['wrk_age']*100

df.drop(columns = ['lf', 'black', 'hisp', 'wrk_age'], inplace = True)

***
# Social Explorer

In [ ]:
land = pd.read_csv('area.csv')
land.head(1)

In [ ]:
land = land[['Geo_FIPS', 'SE_T02A_002']].rename(columns = {'Geo_FIPS': 'fips',
                                                   'SE_T02A_002': 'land'}).set_index('fips')

In [ ]:
df = df.join(land)

In [ ]:
df['density'] = df['pop']/df['land']
df = df.drop(columns = 'land')

************
# Saving

In [ ]:
df['year'] = df.index.get_level_values('year')
df.sort_index(inplace = True)
df.head(1)

In [ ]:
df.isnull().any()
df.shape

In [ ]:
df.dropna(inplace = True)
df.shape

In [ ]:
df.to_pickle('class_data.pkl')
df.info()

In [ ]:
os.listdir()

In [ ]:
%reset -f

In [ ]:
%whos

In [ ]:
import pandas as pd
pd.read_pickle('class_data.pkl')